In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, auc, average_precision_score
#import scikitplot as skplt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import keras.backend as K
from NGF.preprocessing import tensorise_smiles
from custom_layers.model_creator import encode_smiles, stage_creator
from keras.layers import Dense, Dropout, Input, Lambda, concatenate,Flatten
from keras.models import Model, load_model
import dill
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
from math import ceil
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
target_1 = 'p38'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)
    
target_2 = 'akt1'
base_path_2 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_2 = base_path_2+f'/data/{target_2}/data.csv'
df_akt1 = pd.read_csv(data_fpath_2).set_index('biolab_index')

with open(base_path_2+f'/data/{target_2}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_akt1 = dill.load(in_f)
with open(base_path_2+f'/data/{target_2}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_akt1 = dill.load(in_f)
    
target_3 = 'pi3k'
base_path_3 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_3 = base_path_3+f'/data/{target_3}/data.csv'
df_pi3k = pd.read_csv(data_fpath_3).set_index('biolab_index')

with open(base_path_3+f'/data/{target_3}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_pi3k = dill.load(in_f)
with open(base_path_3+f'/data/{target_3}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_pi3k = dill.load(in_f)

In [3]:
train_p38 = df_p38.loc[train_test_folds_p38[0]]            
val_p38 = df_p38.loc[train_test_folds_p38[1]]
                   

train_akt1 = df_akt1.loc[train_test_folds_akt1[0]]           
val_akt1 = df_akt1.loc[train_test_folds_akt1[1]]
                   

train_pi3k = df_pi3k.loc[train_test_folds_pi3k[0]]        
val_pi3k = df_pi3k.loc[train_test_folds_pi3k[1]]

In [4]:
#Random splits with sklearn (on our test set)
df_p38 = df_p38.reset_index(drop=True)
X_train_p38, X_val_p38, Y_train_p38, Y_val_p38 = train_test_split(df_p38.rdkit,
                                                                  df_p38.Binary,
                                                                  test_size = 0.15,
                                                                  train_size = 0.85,
                                                                  shuffle = True)
random_train_p38 = pd.DataFrame(X_train_p38)
random_train_p38['Binary'] = Y_train_p38
random_val_p38 = pd.DataFrame(X_val_p38)
random_val_p38['Binary'] = Y_val_p38
del X_train_p38,X_val_p38,Y_train_p38,Y_val_p38


df_akt1 = df_akt1.reset_index(drop=True)
X_train_akt1, X_val_akt1, Y_train_akt1, Y_val_akt1 = train_test_split(df_akt1.rdkit,
                                                                     df_akt1.Binary,
                                                                     test_size = 0.15,
                                                                     train_size = 0.85,
                                                                     shuffle = True)
random_train_akt1 = pd.DataFrame(X_train_akt1)
random_train_akt1['Binary'] = Y_train_akt1
random_val_akt1 = pd.DataFrame(X_val_akt1)
random_val_akt1['Binary'] = Y_val_akt1
del X_train_akt1,X_val_akt1,Y_train_akt1,Y_val_akt1


df_pi3k = df_pi3k.reset_index(drop=True)
X_train_pi3k, X_val_pi3k, Y_train_pi3k, Y_val_pi3k = train_test_split(df_pi3k.rdkit,
                                                                      df_pi3k.Binary,
                                                                      test_size = 0.15,
                                                                      train_size = 0.85,
                                                                      shuffle = True)
random_train_pi3k = pd.DataFrame(X_train_pi3k)
random_train_pi3k['Binary'] = Y_train_pi3k
random_val_pi3k = pd.DataFrame(X_val_pi3k)
random_val_pi3k['Binary'] = Y_val_pi3k
del X_train_pi3k,X_val_pi3k,Y_train_pi3k,Y_val_pi3k


In [5]:
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.00001)
model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(96), int(104), int(120)],
        "fp_length" : [int(160), int(160), int(160)],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(3*160), int(3*160), int(3*128)], 
        'dropout_rate' : [0.355453731255706, 0.355453731255706],
        'lr' : 0.0007037117031430456,
        'batch_size' : int(256),
        'n_epochs' : int(30),
        }


In [6]:
def csv_generator_MLT(bs,
                      df_p1,df_p2,df_p3,
                      smiles_list_p1,smiles_list_p2,smiles_list_p3,
                      atoms_p1,bonds_p1,edges_p1,
                      atoms_p2,bonds_p2,edges_p2,
                      atoms_p3,bonds_p3,edges_p3,mode = 'train',aug = None):
    counter_1=int(0)
    counter_2=int(0)
    counter_3=int(0)
    #Keep looping indefinetely
    df_p1 = df_p1.reset_index(drop=True)
    df_p2 = df_p2.reset_index(drop=True)
    df_p3 = df_p3.reset_index(drop=True)
    while True:
        
        #Initialize batches of inputs and outputs
        ind1 = []
        ind2 = []
        ind3 = []
        
        d_p1=[]
        d_p2=[]
        d_p3=[]
        
        #Keep looping until we reach batch size
        while len(ind3)<=bs: #doesn't matter if it is smi1 or smi2 since they have the same len
            if counter_3==len(df_p3):
                counter_3=int(0)
                df_p3 = df_p3.sample(frac=1).reset_index(drop=True)
            
            smi_p3 = df_p3['rdkit'][counter_3]
            ind3.append(smiles_list_p3.index(smi_p3))
            d_p3.append(df_p3['Binary'][counter_3])
            counter_3+=1
            while len(ind1)<=bs:
                if counter_1==len(df_p1):
                    counter_1=int(0)
                    df_p1 = df_p1.sample(frac=1).reset_index(drop=True)
                
                smi_p1 = df_p1['rdkit'][counter_1]
                ind1.append(smiles_list_p1.index(smi_p1))
                d_p1.append(df_p1['Binary'][counter_1])
                counter_1+=1
                
                
                while len(ind2)<=bs:
                    if counter_2==len(df_p2):
                        counter_2=int(0)
                        df_p2 = df_p2.sample(frac=1).reset_index(drop=True)
                    
                    smi_p2 = df_p2['rdkit'][counter_2]
                    ind2.append(smiles_list_p2.index(smi_p2))
                    d_p2.append(df_p2['Binary'][counter_2])
                    counter_2+=1
                # if we are evaluating we should now break from our 
                # loop to ensure we don't continue to fill up the batch from samples at the beginning of the file
            
            
        atoms_target_1 = np.array(atoms_p1[ind1],dtype = 'float32')
        bonds_target_1 = np.array(bonds_p1[ind1],dtype = 'float32')
        edges_target_1 = np.array(edges_p1[ind1],dtype = 'int32')
        
        
        atoms_target_2 = np.array(atoms_p2[ind2],dtype = 'float32')
        bonds_target_2 = np.array(bonds_p2[ind2],dtype = 'float32')
        edges_target_2 = np.array(edges_p2[ind2],dtype = 'int32')
        
        
        atoms_target_3 = np.array(atoms_p3[ind3],dtype = 'float32')
        bonds_target_3 = np.array(bonds_p3[ind3],dtype = 'float32')
        edges_target_3 = np.array(edges_p3[ind3],dtype = 'int32')
        
        
        yield ({'atom_inputs_1':atoms_target_1,
                'bond_inputs_1':bonds_target_1,
                'edge_inputs_1':edges_target_1,
                'atom_inputs_2':atoms_target_2,
                'bond_inputs_2':bonds_target_2,
                'edge_inputs_2':edges_target_2,
                'atom_inputs_3':atoms_target_3,
                'bond_inputs_3':bonds_target_3,
                'edge_inputs_3':edges_target_3,},{'Protein_1':np.array(d_p1,dtype = 'int64'),
                                                  'Protein_2':np.array(d_p2,dtype = 'int64'),
                                                  'Protein_3':np.array(d_p3,dtype = 'int64')})

In [7]:
def crop(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
        def func(x):
            if dimension == 0:
                return x[start: end]
            if dimension == 1:
                return x[:, start: end]
            if dimension == 2:
                return x[:, :, start: end]
            if dimension == 3:
                return x[:, :, :, start: end]
            if dimension == 4:
                return x[:, :, :, :, start: end]
        return Lambda(func)
class GCN_MLT(object):

    def __init__(self,  model_params):
        self.model_params = model_params

    def build_encoder(self):
        model_enc_1 = stage_creator(self.model_params, 1, conv=True)[0]
        model_enc_2 = stage_creator(self.model_params, 2, conv=True)[0]
        model_enc_3 = stage_creator(self.model_params, 3, conv=True)[0]

        model_enc_fp_1 = stage_creator(self.model_params, 1, conv=False)[1]
        model_enc_fp_2 = stage_creator(self.model_params, 2, conv=False)[1]
        model_enc_fp_3 = stage_creator(self.model_params, 3, conv=False)[1]

        atoms, bonds, edges = encode_smiles(self.model_params["max_atoms"],
                                            self.model_params["num_atom_features"],
                                            self.model_params["max_degree"],
                                            self.model_params["num_bond_features"])

        graph_conv_1 = model_enc_1([atoms, bonds, edges])
        graph_conv_2 = model_enc_2([graph_conv_1, bonds, edges])
        graph_conv_3 = model_enc_3([graph_conv_2, bonds, edges])

        fingerprint_1 = model_enc_fp_1([graph_conv_1, bonds, edges])
        fingerprint_1 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_1)

        fingerprint_2 = model_enc_fp_2([graph_conv_2, bonds, edges])
        fingerprint_2 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_2)

        fingerprint_3 = model_enc_fp_3([graph_conv_3, bonds, edges])
        fingerprint_3 = Lambda(lambda x: K.sum(x, axis=1), output_shape=lambda s: (s[0], s[2]))(fingerprint_3)

        final_fingerprint = keras.layers.add([fingerprint_1, fingerprint_2, fingerprint_3])

        return Model([atoms, bonds, edges], [final_fingerprint])
    
    def build_model(self, encoder, verbose=True):
        #First Target
        atoms_1 = Input(name='atom_inputs_1',
                      shape=(self.model_params['max_atoms'], self.model_params['num_atom_features']), dtype='float32')
        bonds_1 = Input(name='bond_inputs_1', shape=(
        self.model_params['max_atoms'], self.model_params['max_degree'], self.model_params['num_bond_features']),
                      dtype='float32')
        edges_1 = Input(name='edge_inputs_1', shape=(self.model_params['max_atoms'], self.model_params['max_degree']),
                      dtype='int32')
        encode_drug_1 = encoder([atoms_1, bonds_1, edges_1])
        
        #Second Target
        atoms_2 = Input(name='atom_inputs_2',
                      shape=(self.model_params['max_atoms'], self.model_params['num_atom_features']), dtype='float32')
        bonds_2 = Input(name='bond_inputs_2', shape=(
        self.model_params['max_atoms'], self.model_params['max_degree'], self.model_params['num_bond_features']),
                      dtype='float32')
        edges_2 = Input(name='edge_inputs_2', shape=(self.model_params['max_atoms'], self.model_params['max_degree']),
                      dtype='int32')
        encode_drug_2 = encoder([atoms_2, bonds_2, edges_2])
        
        atoms_3 = Input(name='atom_inputs_3',
                      shape=(self.model_params['max_atoms'], self.model_params['num_atom_features']), dtype='float32')
        bonds_3 = Input(name='bond_inputs_3', shape=(
        self.model_params['max_atoms'], self.model_params['max_degree'], self.model_params['num_bond_features']),
                      dtype='float32')
        edges_3 = Input(name='edge_inputs_3', shape=(self.model_params['max_atoms'], self.model_params['max_degree']),
                      dtype='int32')
        encode_drug_3 = encoder([atoms_3, bonds_3, edges_3])
        
        conc = keras.layers.Concatenate(axis = -1)([encode_drug_1,encode_drug_2,encode_drug_3])
        # Fully connected
        FC1 = Dense(self.model_params["dense_size"][0], activation='relu',kernel_initializer='random_normal')(conc)
        FC2 = Dropout(self.model_params["dropout_rate"][0])(FC1)
        FC2 = Dense(self.model_params["dense_size"][1], activation='relu',kernel_initializer='random_normal')(FC2)
        FC2 = Dropout(self.model_params["dropout_rate"][1])(FC2)
        FC2 = Dense(self.model_params["dense_size"][2], activation='relu',kernel_initializer='random_normal')(FC2)
        
        prot_1 = crop(1,0,int(self.model_params['dense_size'][2]/3))(FC2)
        prot_2 = crop(1,int(self.model_params['dense_size'][2]/3),2*int(self.model_params['dense_size'][2]/3))(FC2) 
        prot_3 = crop(1,2*int(self.model_params['dense_size'][2]/3),3*int(self.model_params['dense_size'][2]/3))(FC2)
        
        pred_1 = Dense(1, activation='sigmoid', kernel_initializer='random_normal',name = 'Protein_1')(prot_1)
        pred_2 = Dense(1, activation='sigmoid', kernel_initializer='random_normal',name = 'Protein_2')(prot_2)
        pred_3 = Dense(1, activation='sigmoid', kernel_initializer='random_normal',name = 'Protein_3')(prot_3)
        gcn_model = Model(inputs=[atoms_1, bonds_1, edges_1,
                                  atoms_2, bonds_2, edges_2,
                                  atoms_3, bonds_3, edges_3], 
                                  outputs= [pred_1, pred_2, pred_3])

        adam = keras.optimizers.Adam(lr=self.model_params["lr"], beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
        gcn_model.compile(optimizer=adam, loss= {'Protein_1':'binary_crossentropy',
                                                 'Protein_2':'binary_crossentropy',
                                                 'Protein_3':'binary_crossentropy'}, 
                          metrics = {'Protein_1': keras.metrics.binary_accuracy,
                                     'Protein_2': keras.metrics.binary_accuracy,
                                     'Protein_3': keras.metrics.binary_accuracy})

        if verbose:
            #print('encoder')
            #encoder.summary()
            print('GCN_model')
            gcn_model.summary()

        return gcn_model

    def dataframe_to_gcn_input(self, input_data):
        x_atoms_cold, x_bonds_cold, x_edges_cold = tensorise_smiles(input_data['rdkit'],
                                                                    max_degree=self.model_params['max_degree'],
                                                                    max_atoms=self.model_params['max_atoms'])
        return [x_atoms_cold, x_bonds_cold, x_edges_cold]

In [8]:
training_p38 = [train_p38,random_train_p38]
validation_p38 = [val_p38,random_val_p38]

training_akt1 = [train_akt1,random_train_akt1]
validation_akt1 = [val_akt1,random_val_akt1]

training_pi3k = [train_pi3k,random_train_pi3k]
validation_pi3k = [val_pi3k,random_val_pi3k]

In [9]:
eval_p38 = {}
eval_akt1 = {}
eval_pi3k = {}
gcn = GCN_MLT(model_params)
es2 = EarlyStopping(monitor='loss',patience=15, min_delta=0)
rlr2 = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=2, verbose=1, min_lr=0.00001)
for i in range(len(training_p38)):
    
        #First Target
        X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38 = gcn.dataframe_to_gcn_input(validation_p38[i])
        Y_cold_p38 = validation_p38[i].Binary
        
        X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38 = gcn.dataframe_to_gcn_input(training_p38[i])
        Y_p38 = training_p38[i].Binary
        smiles_list_p38 = list(training_p38[i]['rdkit'])
        
        #Second Target
        X_atoms_cold_akt1, X_bonds_cold_akt1, X_edges_cold_akt1 = gcn.dataframe_to_gcn_input(validation_akt1[i])
        Y_cold_akt1 = validation_akt1[i].Binary
        
        X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1 = gcn.dataframe_to_gcn_input(training_akt1[i])
        Y_akt1 = training_akt1[i].Binary
        
        smiles_list_akt1 = list(training_akt1[i]['rdkit'])
        
        #Third Target
        X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k = gcn.dataframe_to_gcn_input(validation_pi3k[i])
        Y_cold_pi3k = validation_pi3k[i].Binary
        
        X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k = gcn.dataframe_to_gcn_input(training_pi3k[i])
        Y_pi3k = training_pi3k[i].Binary
        
        smiles_list_pi3k = list(training_pi3k[i]['rdkit'])
        
        gcn_encoder = gcn.build_encoder()
        gcn_model = gcn.build_model(gcn_encoder)
        train_GEN = csv_generator_MLT(model_params['batch_size'],
                                         training_p38[i],training_akt1[i],training_pi3k[i],
                                         smiles_list_p38,smiles_list_akt1,smiles_list_pi3k,
                                         X_atoms_train_p38, X_bonds_train_p38, X_edges_train_p38,
                                         X_atoms_train_akt1, X_bonds_train_akt1, X_edges_train_akt1,
                                         X_atoms_train_pi3k, X_bonds_train_pi3k, X_edges_train_pi3k)
        NUM_TRAIN = np.sum([len(training_p38[i]),len(training_akt1[i]),len(training_pi3k[i])])
        gcn_model.fit_generator(train_GEN,
                      steps_per_epoch = ceil(3*NUM_TRAIN/model_params['batch_size']),
                      epochs = 25,
                      shuffle = True,
                      validation_data = None,
                      callbacks = [es2,rlr2])
        Y_pred_train_p38,t1,t2 = gcn_model.predict([X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38,
                                               X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38,
                                               X_atoms_train_p38,X_bonds_train_p38,X_edges_train_p38])
        t1,Y_pred_train_akt1,t2 = gcn_model.predict([X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1,
                                               X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1,
                                               X_atoms_train_akt1,X_bonds_train_akt1,X_edges_train_akt1])
        t1,t2,Y_pred_train_pi3k = gcn_model.predict([X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k,
                                               X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k,
                                               X_atoms_train_pi3k,X_bonds_train_pi3k,X_edges_train_pi3k])


        Y_pred_val_p38,t1,t2 = gcn_model.predict([X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38,
                                            X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38,
                                            X_atoms_cold_p38,X_bonds_cold_p38,X_edges_cold_p38])
        t1,Y_pred_val_akt1,t2 = gcn_model.predict([X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1,
                                            X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1,
                                            X_atoms_cold_akt1,X_bonds_cold_akt1,X_edges_cold_akt1])
        t1,t2,Y_pred_val_pi3k = gcn_model.predict([X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k,
                                            X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k,
                                            X_atoms_cold_pi3k,X_bonds_cold_pi3k,X_edges_cold_pi3k])
        del t1,t2
        
        if i == 0:
            eval_p38['Test'] = calculate_metrics(np.array(Y_cold_p38),Y_pred_val_p38.squeeze())
            eval_akt1['Test'] = calculate_metrics(np.array(Y_cold_akt1),Y_pred_val_akt1.squeeze())
            eval_pi3k['Test'] = calculate_metrics(np.array(Y_cold_pi3k),Y_pred_val_pi3k.squeeze())
        elif i == 1:
            eval_p38['Random'] = calculate_metrics(np.array(Y_cold_p38),Y_pred_val_p38.squeeze())
            eval_akt1['Random'] = calculate_metrics(np.array(Y_cold_akt1),Y_pred_val_akt1.squeeze())
            eval_pi3k['Random'] = calculate_metrics(np.array(Y_cold_pi3k),Y_pred_val_pi3k.squeeze())
        

LAYER 0
LAYER 1
LAYER 2
GCN_model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs_1 (InputLayer)      (None, 70, 62)       0                                            
__________________________________________________________________________________________________
bond_inputs_1 (InputLayer)      (None, 70, 5, 6)     0                                            
__________________________________________________________________________________________________
edge_inputs_1 (InputLayer)      (None, 70, 5)        0                                            
__________________________________________________________________________________________________
atom_inputs_2 (InputLayer)      (None, 70, 62)       0                                            
___________________________________________________________________________

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
95/95 [==============================] - 21s 226ms/step - loss: 1.8789 - Protein_1_loss: 0.6591 - Protein_2_loss: 0.5560 - Protein_3_loss: 0.6637 - Protein_1_binary_accuracy: 0.5931 - Protein_2_binary_accuracy: 0.7001 - Protein_3_binary_accuracy: 0.6240
Epoch 2/25
95/95 [==============================] - 13s 136ms/step - loss: 1.4924 - Protein_1_loss: 0.4874 - Protein_2_loss: 0.3878 - Protein_3_loss: 0.6172 - Protein_1_binary_accuracy: 0.7597 - Protein_2_binary_accuracy: 0.8308 - Protein_3_binary_accuracy: 0.6551
Epoch 3/25
95/95 [==============================] - 13s 137ms/step - loss: 1.2615 - Protein_1_loss: 0.4102 - Protein_2_loss: 0.3545 - Protein_3_loss: 0.4968 - Protein_1_binary_accuracy: 0.8089 - Protein_2_binary_accuracy: 0.8464 - Protein_3_binary_accuracy: 0.7578
Epoch 4/25
95/95 [==============================] - 13s 136ms/step - loss: 1.1336 - Protein_1_loss: 0.3766 - Protein_2_loss: 0.3224 - Protein_3_loss: 0.4346 - Protein_1_binary_accuracy: 0.8300 - Protein_2_

C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
95/95 [==============================] - 21s 222ms/step - loss: 1.8832 - Protein_1_loss: 0.6429 - Protein_2_loss: 0.5946 - Protein_3_loss: 0.6457 - Protein_1_binary_accuracy: 0.6169 - Protein_2_binary_accuracy: 0.6632 - Protein_3_binary_accuracy: 0.6362
Epoch 2/25
95/95 [==============================] - 13s 136ms/step - loss: 1.3894 - Protein_1_loss: 0.4727 - Protein_2_loss: 0.4053 - Protein_3_loss: 0.5114 - Protein_1_binary_accuracy: 0.7726 - Protein_2_binary_accuracy: 0.8160 - Protein_3_binary_accuracy: 0.7406
Epoch 3/25
95/95 [==============================] - 13s 137ms/step - loss: 1.1899 - Protein_1_loss: 0.4104 - Protein_2_loss: 0.3508 - Protein_3_loss: 0.4287 - Protein_1_binary_accuracy: 0.8095 - Protein_2_binary_accuracy: 0.8473 - Protein_3_binary_accuracy: 0.7943
Epoch 4/25
95/95 [==============================] - 13s 136ms/step - loss: 1.0953 - Protein_1_loss: 0.3791 - Protein_2_loss: 0.3104 - Protein_3_loss: 0.4058 - Protein_1_binary_accuracy: 0.8263 - Protein_2_

In [10]:
eval_p38 = pd.DataFrame(eval_p38).T
eval_p38.to_csv('../../../../Desktop/binding/thesis english/Results/4-Multitask/GCN/p38.csv')

eval_akt1 = pd.DataFrame(eval_akt1).T
eval_akt1.to_csv('../../../../Desktop/binding/thesis english/Results/4-Multitask/GCN/akt1.csv')

eval_pi3k = pd.DataFrame(eval_pi3k).T
eval_pi3k.to_csv('../../../../Desktop/binding/thesis english/Results/4-Multitask/GCN/pi3k.csv')

In [11]:
eval_p38

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Test,0.820632,178.0,128.0,25.0,178.0,0.735594,0.581699,0.876847,0.699411
Random,0.908345,238.0,43.0,43.0,210.0,0.889229,0.830040,0.830040,0.838951


In [12]:
eval_akt1

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Test,0.852450,147.0,43.0,29.0,87.0,0.786455,0.669231,0.750000,0.764706
Random,0.882756,159.0,27.0,36.0,99.0,0.842274,0.785714,0.733333,0.803738


In [13]:
eval_pi3k

,roc_auc,tn,fp,fn,tp,map,precision,recall,accuracy
Test,0.893848,325.0,35.0,59.0,118.0,0.773824,0.771242,0.666667,0.824953
Random,0.863630,284.0,74.0,53.0,153.0,0.801899,0.674009,0.742718,0.774823
